# OpenAI Supervised Fine-Tuning

This recipe allows TensorZero users to fine-tune OpenAI models using their own data.
Since TensorZero automatically logs all inferences and feedback, it is straightforward to fine-tune a model using your own data and any prompt you want.

To use this recipe, you need to set the following parameters:


In [1]:
from pathlib import Path

CLICKHOUSE_HOST = "localhost"
METRIC_NAME = "haiku_score"
FUNCTION_NAME = "write_haiku"
# The name of the variant to use to grab the templates used for fine-tuning
TEMPLATE_VARIANT_NAME = "initial_prompt_gpt4o"
CONFIG_PATH = Path("../../examples/haiku_hidden_preferences/config/tensorzero.toml")
# Only relevant if the metric is a float metric
FLOAT_METRIC_THRESHOLD = 0.5
# Max number of samples to use for fine-tuning
MAX_SAMPLES = 100_000
# The name of the model to fine-tune (supported models [here](https://platform.openai.com/docs/guides/fine-tuning))
MODEL_NAME = "gpt-4o-mini-2024-07-18"
DATABASE_NAME = "tensorzero"

In [24]:
import json
import tempfile
import time
from typing import Any, Dict, List

import openai
import toml
from clickhouse_driver import Client
from IPython.display import clear_output
from minijinja import Environment

In [3]:
with CONFIG_PATH.open("r") as f:
    config = toml.load(f)

In [4]:
metric = config["metrics"][METRIC_NAME]
if metric["type"] not in ["float", "boolean"]:
    raise ValueError(
        f"Metric {METRIC_NAME} is not a float or boolean metric and is not supported by this recipe."
    )

In [5]:
variant = config["functions"][FUNCTION_NAME]["variants"][TEMPLATE_VARIANT_NAME]
config_dir = CONFIG_PATH.parent
system_template_path = (
    config_dir / variant["system_template"] if "system_template" in variant else None
)
user_template_path = (
    config_dir / variant["user_template"] if "user_template" in variant else None
)
assistant_template_path = (
    config_dir / variant["assistant_template"]
    if "assistant_template" in variant
    else None
)

In [6]:
variant

{'type': 'chat_completion',
 'weight': 0.25,
 'model': 'gpt-4o-2024-08-06',
 'system_template': 'functions/write_haiku/initial_prompt/system_template.minijinja',
 'user_template': 'functions/write_haiku/initial_prompt/user_template.minijinja'}

In [7]:
templates = {}
if system_template_path:
    with system_template_path.open("r") as f:
        templates["system"] = f.read()
if user_template_path:
    with user_template_path.open("r") as f:
        templates["user"] = f.read()
if assistant_template_path:
    with assistant_template_path.open("r") as f:
        templates["assistant"] = f.read()
env = Environment(templates=templates)

In [8]:
clickhouse_client = Client(host=CLICKHOUSE_HOST, database=DATABASE_NAME)

In [9]:
metric_table_name = {
    "float": "FloatMetricFeedback",
    "boolean": "BooleanMetricFeedback",
}.get(metric["type"])

if metric_table_name is None:
    raise ValueError(f"Unsupported metric type: {metric['type']}")

In [10]:
# Query the inferences and feedback from the database and join them on the inference id
df = clickhouse_client.query_dataframe(f"""SELECT 
    i.variant_name, 
    i.input, 
    i.output, 
    f.value
FROM 
    Inference i
JOIN 
    {metric_table_name} f ON i.id = f.target_id
WHERE 
    i.function_name = '{FUNCTION_NAME}'""")

In [12]:
# We filter the dataframe to only include "good" examples
threshold = FLOAT_METRIC_THRESHOLD if metric["type"] == "float" else 0.5
optimize_direction = metric["optimize"]
filtered_df = (
    df[df["value"] > threshold]
    if optimize_direction == "max"
    else df[df["value"] < threshold].copy()
)
print("The original dataframe has", len(df), "rows")
print("The filtered dataframe has", len(filtered_df), "rows")

The original dataframe has 500 rows
The filtered dataframe has 243 rows


In [13]:
def render_message(content: List[Dict[str, Any]], role: str) -> str:
    assert role in ["user", "assistant"]
    if len(content) != 1:
        raise ValueError(f"Message {content} has a not-one number of content blocks.")
    if content[0]["type"] != "text":
        raise ValueError(f"Message {content} has a not-text content block.")
    content = content[0]["value"]
    if isinstance(content, str):
        return content
    else:
        return env.render_template(role, **content)

In [14]:
def example_to_openai_messages(example) -> List[Dict[str, str]]:
    function_input = json.loads(example["input"])
    system = function_input.get("system", {})
    rendered_messages = []
    # Add the system message to the rendered messages
    # If there is data passed in or a system template there must be a system message
    if len(system) > 0 or system_template_path:
        if system_template_path:
            system_message = env.render_template("system", **system)
            rendered_messages.append({"role": "system", "content": system_message})
        else:
            rendered_messages.append({"role": "system", "content": system})
    # Add the input messages to the rendered messages
    for message in function_input["messages"]:
        rendered_message = render_message(message["content"], message["role"])
        rendered_messages.append({"role": message["role"], "content": rendered_message})
    # Add the output to the messages
    output = json.loads(example["output"])
    if len(output) != 1:
        raise ValueError(f"Output {output} has a not-one number of content blocks.")
    if output[0]["type"] != "text":
        raise ValueError(f"Output {output} has a not-text content block.")
    rendered_messages.append({"role": "assistant", "content": output[0]["text"]})
    return dict(messages=rendered_messages)

In [15]:
filtered_df["openai_messages"] = filtered_df.apply(example_to_openai_messages, axis=1)

/var/folders/wn/t71_3fwd01d9tfdyt_gcnmv40000gn/T/ipykernel_8232/3606034244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["openai_messages"] = filtered_df.apply(example_to_openai_messages, axis=1)


In [22]:
# TODO: do a train-val split, upload both to OpenAI and use the val set as part of the job
# Create a temporary file in the format that OpenAI expects for fine-tuning
temp_file = tempfile.NamedTemporaryFile(mode="w", suffix=".jsonl")

# Write the openai_messages to the temporary file
for item in filtered_df["openai_messages"]:
    json.dump(item, temp_file)
    temp_file.write("\n")
temp_file.flush()
# Get the file path
temp_file_path = temp_file.name
print(f"temp_file_path: {temp_file_path}")
# Be sure to close the file later to avoid a resource leak

temp_file_path: /var/folders/wn/t71_3fwd01d9tfdyt_gcnmv40000gn/T/tmpvyeki17k.jsonl


In [17]:
openai_client = openai.OpenAI()
# Upload the file to OpenAI
file_object = openai_client.files.create(
    file=open(temp_file_path, "rb"), purpose="fine-tune"
)
# We can close it now (deletes but we've already uploaded)
temp_file.close()

In [18]:
ft_job = openai_client.fine_tuning.jobs.create(
    training_file=file_object.id, model=MODEL_NAME
)

In [25]:
# Keep an eye on the job status (this will take a while)
while True:
    clear_output(wait=True)
    job_status = openai_client.fine_tuning.jobs.retrieve(ft_job.id)
    print(f"job_status: {job_status}")
    if job_status.status in ("succeeded", "failed", "cancelled"):
        break
    time.sleep(10)

job_status: FineTuningJob(id='ftjob-4h6UuiWHgwJ6dTpWXXBzoRS1', created_at=1724776316, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-fewHWgmYjDeYGco5co60C7fh', result_files=[], seed=1523924585, status='running', trained_tokens=None, training_file='file-jFppbsRqj4QNTVQK41dVXsXR', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
job_status: FineTuningJob(id='ftjob-4h6UuiWHgwJ6dTpWXXBzoRS1', created_at=1724776316, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-fewHWgmYjDeYGco5co60C7fh', result_files=[], seed=1523924585, status='running', trained_t

In [32]:
fine_tuned_model = job_status.fine_tuned_model
model_instructions = f"""Add the following block to your config file to include the fine-tuned model in your gateway:
```toml
[models."{fine_tuned_model}"]
routing = ["openai"]

[models."{fine_tuned_model}".providers.openai]
type = "openai"
model_name = "{fine_tuned_model}"
```
"""

print(model_instructions)

Add the following block to your config file to include the fine-tuned model in your gateway:
```toml
[models."ft:gpt-4o-2024-08-06:tensorzero::A0tlZdHo"]
routing = ["openai"]

[models."ft:gpt-4o-2024-08-06:tensorzero::A0tlZdHo".providers.openai]
type = "openai"
model_name = "ft:gpt-4o-2024-08-06:tensorzero::A0tlZdHo"
```



In [36]:
variant_instructions = f"""Add the following block to your config file to include the fine-tuned model in your function:
```toml
[functions."{FUNCTION_NAME}".variants.YOUR_VARIANT_NAME]
weight = 0
"""
system_template = variant.get("system_template")
if system_template:
    variant_instructions += f"""system_template = "{system_template}"
"""
user_template = variant.get("user_template")
if user_template:
    variant_instructions += f"""user_template = "{user_template}"
"""
assistant_template = variant.get("assistant_template")
if assistant_template:
    variant_instructions += f"""assistant_template = "{assistant_template}"
"""
variant_instructions += "OTHER PARAMETERS (max_tokens, temperature, etc.) HERE\n```"
print(variant_instructions)
print("You can change the weight to enable a gradual rollout of the new model.")

Add the following block to your config file to include the fine-tuned model in your function:
```toml
[functions."write_haiku".variants.YOUR_VARIANT_NAME]
weight = 0
system_template = "functions/write_haiku/initial_prompt/system_template.minijinja"
user_template = "functions/write_haiku/initial_prompt/user_template.minijinja"
OTHER PARAMETERS (max_tokens, temperature, etc.) HERE
```
You can change the weight to enable a gradual rollout of the new model.
